# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 35

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# import player data 
filepath = '../data/fbref/player_stats_week' + str(latest_gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)
playerStats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,90s,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Matches
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,30-258,1990,21,20,1774,19.7,0,1,0,0,0,2,0,0.00,0.05,0.05,0.00,0.05,1.2,1.2,0.8,2.0,0.06,0.04,0.10,0.06,0.10,Matches
2,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,23-224,1997,22,12,1040,11.6,6,1,6,0,0,0,0,0.52,0.09,0.61,0.52,0.61,6.4,6.4,0.9,7.3,0.56,0.07,0.63,0.56,0.63,Matches
3,Che Adams\Che-Adams,sco SCO,FW,Southampton,24-305,1996,34,28,2532,28.1,8,5,8,0,0,1,0,0.28,0.18,0.46,0.28,0.46,8.2,8.2,5.4,13.5,0.29,0.19,0.48,0.29,0.48,Matches
4,Tosin Adarabioyo\Tosin-Adarabioyo,eng ENG,DF,Fulham,23-232,1997,30,30,2683,29.8,0,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,1.1,1.1,0.3,1.4,0.04,0.01,0.05,0.04,0.05,Matches
5,Adrián\Adrian,es ESP,GK,Liverpool,34-131,1987,3,3,270,3.0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [4]:
# import team data and pre-process
filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col='Squad')
teamStats.head()

,Rk,MP,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Squad,,,,,,,,,,,,,,,,,,
Manchester City,1,35,25,5,5,72,26,46,80,68.4,26.5,41.9,1.20,W L W W L,NaN,İlkay Gündoğan - 12,Ederson,NaN
Manchester Utd,2,36,20,10,6,70,42,28,70,55.4,38.0,17.3,0.50,W D W L L,NaN,Bruno Fernandes - 18,David de Gea,NaN
Leicester City,3,36,20,6,10,65,44,21,66,53.3,43.0,10.3,0.29,W W D L W,NaN,Jamie Vardy - 13,Kasper Schmeichel,NaN
Chelsea,4,36,18,10,8,55,33,22,64,58.8,31.0,27.8,0.77,D W W W L,111.0,Timo Werner Mason Mount... - 6,Edouard Mendy,NaN
Liverpool,5,35,17,9,9,61,41,20,60,62.9,41.1,21.8,0.64,W D D W W,333.0,Mohamed Salah - 21,Alisson,NaN


In [5]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)
# this is used later
df_online=1

# fetch data locally
#df = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek) + '.csv', index_col=0)
#df_online=0

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0,37605,NaN,,0,0,-3,3,0.0,703,288,NaN,,0,3,0.0,0.0,0,Mesut,0.0,0,0,0.0,703,288,False,0.0,703,288,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,67,0,0,NaN,0,,37605.jpg,0.0,0,0,Özil,0.5,False,None,u,1,3,0.0,703,288,0,3441,0,54896,22,0.0,0.0,Özil,0
2,0,0,0,0.0,0.0,0,39476,NaN,,0,0,-2,2,0.0,664,239,NaN,,0,2,0.0,0.0,0,Sokratis,0.0,0,0,0.0,675,238,False,0.0,672,238,0,Left the club by mutual consent on 20/1,2020-10-21T10:30:18.546407Z,48,0,0,NaN,0,,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,None,u,1,3,0.0,656,236,0,10266,0,19137,3,0.0,0.0,Sokratis,0
3,0,0,230,75.0,75.0,3,41270,NaN,,0,0,-1,1,46.7,341,115,4.0,,0,2,0.0,1.0,0,David,0.2,20,1,43.9,298,100,False,250.6,253,97,1396,Hamstring injury - 75% chance of playing,2021-05-02T16:00:26.134258Z,54,0,0,NaN,0,,41270.jpg,2.0,1,0,Luiz Moreira Marinho,0.8,False,None,d,1,3,144.0,221,60,41,77114,35,130230,444,0.0,7.6,David Luiz,1
4,3,11,350,100.0,100.0,9,54694,NaN,,0,0,-7,7,323.0,111,75,NaN,,3,3,0.0,8.2,6,Pierre-Emerick,3.6,25,10,159.5,50,30,False,489.8,110,40,2162,,2021-04-12T08:30:26.681398Z,113,1,0,1.0,0,,54694.jpg,4.7,0,0,Aubameyang,6.8,False,None,a,1,3,785.0,34,18,126,1185608,3176,3851394,21455,0.3,11.2,Aubameyang,2
5,1,3,125,100.0,100.0,2,58822,NaN,,0,0,-4,4,114.8,238,60,5.0,,0,2,0.0,1.0,0,Cédric,0.0,11,0,29.2,345,123,False,110.8,355,135,744,,2020-09-23T09:00:14.881983Z,46,0,0,NaN,0,,58822.jpg,2.8,0,0,Soares,0.3,False,None,a,1,3,66.0,319,106,28,41293,62,65475,837,0.0,6.1,Cédric,1


In [6]:
# assign proper team names for each player
team_names = np.sort(teamStats.index)
# for some reason the fpl team numbers are not alphabetical with leeds and leicester...
if (team_names[8]=='Leeds United') & (team_names[9]=='Leicester City'):
    team_names[8] = 'Leicester City'
    team_names[9] = 'Leeds United'
df['team_name'] = team_names[df['team']-1]

## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [7]:
teamStats['xG per game'] = teamStats['xG'] / teamStats['MP']
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Rk,MP,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Manchester City,1,35,25,5,5,72,26,46,80,68.4,26.5,41.9,1.20,W L W W L,NaN,İlkay Gündoğan - 12,Ederson,NaN,1.954286,0.757143,0.469005
Manchester Utd,2,36,20,10,6,70,42,28,70,55.4,38.0,17.3,0.50,W D W L L,NaN,Bruno Fernandes - 18,David de Gea,NaN,1.538889,1.055556,0.347999
Leicester City,3,36,20,6,10,65,44,21,66,53.3,43.0,10.3,0.29,W W D L W,NaN,Jamie Vardy - 13,Kasper Schmeichel,NaN,1.480556,1.194444,0.302872
Chelsea,4,36,18,10,8,55,33,22,64,58.8,31.0,27.8,0.77,D W W W L,111.0,Timo Werner Mason Mount... - 6,Edouard Mendy,NaN,1.633333,0.861111,0.422692
Liverpool,5,35,17,9,9,61,41,20,60,62.9,41.1,21.8,0.64,W D D W W,333.0,Mohamed Salah - 21,Alisson,NaN,1.797143,1.174286,0.309040
West Ham,6,35,17,7,11,55,45,10,58,47.5,44.8,2.7,0.08,W L L W L,111.0,Tomáš Souček Michail Antonio... - 9,Łukasz Fabiański,NaN,1.357143,1.280000,0.278037
Tottenham,7,35,16,8,11,61,41,20,56,48.5,44.5,4.0,0.11,L D W W L,118.0,Harry Kane - 21,Hugo Lloris,NaN,1.385714,1.271429,0.280431
Everton,8,35,16,8,11,46,42,4,56,42.7,45.9,-3.1,-0.09,D W L W D,118.0,Dominic Calvert-Lewin - 16,Jordan Pickford,NaN,1.220000,1.311429,0.269435
Arsenal,9,36,16,7,13,50,38,12,55,51.1,42.9,8.2,0.23,D L W W W,111.0,Alexandre Lacazette - 13,Bernd Leno,NaN,1.419444,1.191667,0.303715


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players' assists based on their xA.

In [8]:
# auxiliary information and variables
df['points_per_game'] = df['points_per_game'].astype(float)
df['games played'] = df['total_points']/df['points_per_game']
xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
xA_proportion

0.7360144057623048

In [9]:
df_previous_week = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek-1) + '.csv', index_col=0)
if latest_gameweek == 1:
    df_previous_week['minutes'] = 0
    df_previous_week['xG'] = np.nan
    df_previous_week['xA'] = np.nan
df_previous_week.head()

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,assists_week16,assists_week17,assists_week18,assists_week19,assists_week2,assists_week20,assists_week21,assists_week22,assists_week23,assists_week24,assists_week25,assists_week26,assists_week27,assists_week28,assists_week29,assists_week3,assists_week30,assists_week31,assists_week32,assists_week33,assists_week34,assists_week4,assists_week5,assists_week6,assists_week7,assists_week8,assists_week9,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheet_points,clean_sheets,cleansheet_week1,cleansheet_week10,cleansheet_week11,cleansheet_week12,cleansheet_week13,cleansheet_week14,cleansheet_week15,...,xG_week28,xG_week29,xG_week3,xG_week30,xG_week31,xG_week32,xG_week33,xG_week34,xG_week4,xG_week5,xG_week6,xG_week7,xG_week8,xG_week9,xPoints,xPoints week 1,xPoints week 10,xPoints week 11,xPoints week 12,xPoints week 13,xPoints week 14,xPoints week 15,xPoints week 16,xPoints week 17,xPoints week 18,xPoints week 19,xPoints week 2,xPoints week 20,xPoints week 21,xPoints week 22,xPoints week 23,xPoints week 24,xPoints week 25,xPoints week 26,xPoints week 27,xPoints week 28,xPoints week 29,xPoints week 3,xPoints week 30,xPoints week 31,xPoints week 32,xPoints week 33,xPoints week 34,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,55.185182,2.691960,0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0,230,50.0,100.0,25.585182,3,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.1,0.0,0.1,0.0,NaN,NaN,NaN,1.6000,NaN,0.000000,NaN,NaN,1.000000,3.397316,NaN,NaN,NaN,2.471518,7.363273,1.00000,3.204777,3.586388,-3.000000,NaN,4.395247,1.000000,5.026123,2.807586,3.986341,1.401035,1.199148,NaN,NaN,NaN,NaN,1.6000,5.274923,3.586388,1.000000,NaN,NaN,NaN,1
4,121.734670,4.666496,2,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,11,332,100.0,100.0,8.139507,8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,1.2,0.0,0.0,0.3,0.9,0.0,0.3,10.5184,3.818731,4.246597,3.606531,1.70657,7.132871,NaN,NaN,5.449329,6.67032,2.867879,11.740818,5.61799,NaN,NaN,1.500000,1.500000,15.348812,2.122456,1.500000,7.201897,NaN,2.100259,2.049787,3.010803,NaN,NaN,NaN,10.5184,2.818731,3.746597,4.101194,7.540818,2.765299,3.282085,2
5,34.158159,3.415816,1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,125,100.0,100.0,12.480577,2,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,10.658585,NaN,3.804777,2.586388,1.986388,3.090127,4.095247,NaN,4.426123,NaN,3.986341,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [10]:
if latest_gameweek > 1:
    filepath = '../data/fbref/team_stats_week' + str(latest_gameweek-1) + '.csv'
    teamStats_previous_week = pd.read_csv(filepath, index_col='Squad')
    display(teamStats_previous_week.head())
elif latest_gameweek == 1:
    teamStats_previous_week = pd.DataFrame(np.nan, index=team_names, columns=['xGA'])
    display(teamStats_previous_week)

,Rk,MP,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Manchester City,1,34,25,5,4,71,24,47,80,66.2,25.7,40.5,1.19,W W L W W,NaN,İlkay Gündoğan - 12,Ederson,NaN,1.947059,0.755882,0.469596
Manchester Utd,2,33,19,10,4,64,35,29,67,53.2,36.8,16.4,0.50,W W W W D,NaN,Bruno Fernandes - 16,David de Gea,NaN,1.612121,1.115152,0.327866
Leicester City,3,34,19,6,9,61,39,22,63,50.3,40.0,10.3,0.30,L L W W D,NaN,Jamie Vardy - 13,Kasper Schmeichel,NaN,1.479412,1.176471,0.308365
Chelsea,4,34,17,10,7,53,31,22,61,55.9,27.9,28.0,0.82,L W D W W,118.0,Timo Werner Tammy Abraham... - 6,Edouard Mendy,NaN,1.644118,0.820588,0.440173
West Ham,5,34,17,7,10,55,44,11,58,46.4,43.1,3.3,0.10,W W L L W,118.0,Tomáš Souček Michail Antonio... - 9,Łukasz Fabiański,NaN,1.364706,1.267647,0.281493


In [11]:
def points_for_goal(position):
    if position < 3:
        return 6
    elif position==3:
        return 5
    elif position==4:
        return 4

def points_for_clean_sheet(position):
    if position < 3:
        return 4
    elif position==3:
        return 1
    elif position==4:
        return 0

In [12]:
# column names to be used below
xPoints_column = 'xPoints week ' + str(latest_gameweek)
xG_column = 'xG_week' + str(latest_gameweek) 
goals_column = 'goals_week' + str(latest_gameweek) 
xGA_column = 'xGA_week' + str(latest_gameweek) 
cleansheet_column = 'cleansheet_week' + str(latest_gameweek) 
xA_column = 'xA_week' + str(latest_gameweek) 
assists_column = 'assists_week' + str(latest_gameweek) 

In [13]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, 'xG'].values[0]  
    penalty_attempts = playerStats.loc[indicator,'PKatt'].values[0]
    position = df.loc[ix, 'element_type']
    
    # 0.24 is the probability to miss a penalty, which incurs -2 points
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] \
                                        - points_for_goal(position) * (df.loc[ix, 'goals_scored'] - xG) \
                                        - 2 * 0.24 * penalty_attempts    
    df.loc[ix, 'xG'] = xG
    df.loc[ix, 'xG_points'] = points_for_goal(position)*xG - 2 * 0.24 * penalty_attempts
    
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xG']):
                df.loc[ix, xG_column] = df.loc[ix, 'xG'] - df_previous_week.loc[ix, 'xG']
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored'] - df_previous_week.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
            else:
                df.loc[ix, xG_column] = xG
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xG)')

In [14]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[team, 'probability no goals allowed']
    position = df.loc[ix, 'element_type']
    
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] - points_for_clean_sheet(position) * \
                                ( df.loc[ix, 'clean_sheets'] - probability_cleanSheet*df.loc[ix, 'games played'] )
    df.loc[ix, 'clean_sheet_points'] = points_for_clean_sheet(position) * \
                                       df.loc[ix, 'games played'] * probability_cleanSheet
    try:
        if ~np.isnan(teamStats_previous_week.loc[team, 'xGA']):            
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA'] - teamStats_previous_week.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets'] - df_previous_week.loc[ix, 'clean_sheets']
        else:
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets']
        
        if df.loc[ix,'minutes'] - df_previous_week.loc[ix,'minutes'] >= 60:
            probability_cleansheet_thisweek = poisson.pmf(0,df.loc[ix, xGA_column])
            df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - points_for_clean_sheet(position) * \
                                            (df.loc[ix, cleansheet_column] - probability_cleansheet_thisweek)
        
    except KeyError:  
        print(str(ix) + ' is a new index. (xGA)')

In [15]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, 'xA'].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)
    df.loc[ix, 'xA'] = xA
    df.loc[ix, 'xA_points'] = 3*xA
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xA']):
                df.loc[ix, xA_column] = df.loc[ix, 'xA'] - df_previous_week.loc[ix, 'xA']
                df.loc[ix, assists_column] = df.loc[ix, 'assists'] - df_previous_week.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                    3 * (xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
            else:
                df.loc[ix, xA_column] = xA
                df.loc[ix, assists_column] = df.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                            3*(xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xA)')

In [16]:
def calculateAdjustedPoints(indicator, index):
    incorporate_xG(indicator, index)
    team_xGA(indicator, index)
    xA(indicator, index)

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [17]:
for ix in df[df['minutes']>0].index: 
    # player name in FPL data
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    # find FBREF data indexes where player name contains 'name'
    indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains(name)
    # if unique match is found, we can calculate 'adjusted points'
    if playerStats.loc[indicator].shape[0]==1:
        calculateAdjustedPoints(indicator, ix)
    # if no match is found
    elif playerStats.loc[indicator].shape[0]==0:
        # try matching based on first name, helps in some cases
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[first_name_indicator].shape[0]==1:
            calculateAdjustedPoints(first_name_indicator, ix)
        # here we deal with some special cases individually
        else:
            names = ['rodrigo','elliott','mattylongstaff',"n'lundulu",'carlosvinicius']
            if name == 'rodrigo':
                exceptional_case_indicator = \
                (playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodri')) & \
                (~playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodriguez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'elliott':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('elliot'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'mattylongstaff':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.contains('matthew longstaff'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == "n'lundulu":
                exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('lundulu')
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'carlosvinicius':
                exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('carlos')
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            else:
                print(str(ix) + ': no player found.')
    # if more than one matches were found in the original comparison
    elif playerStats.loc[indicator].shape[0]>1:
        # try matching full name
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[full_name_indicator].shape[0]==1:
            calculateAdjustedPoints(full_name_indicator, ix)
        # here we try to match players based both on their name and team
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats['Squad']==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                calculateAdjustedPoints(indicator & team_indicator, ix)
            # special case
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('heung')
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

702 is a new index. (xG)
702 is a new index. (xGA)
702 is a new index. (xA)
699 is a new index. (xG)
699 is a new index. (xGA)
699 is a new index. (xA)


In [18]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

if (latest_gameweek > 1) & (df_online==1):
    column_list = []
    for i in range(1,latest_gameweek):
        column_list.append('xPoints week ' + str(i))
        column_list.append('xG_week' + str(i))
        column_list.append('goals_week' + str(i))
        column_list.append('xGA_week' + str(i))
        column_list.append('cleansheet_week' + str(i))
        column_list.append('xA_week' + str(i))
        column_list.append('assists_week' + str(i))
    df = df.join( df_previous_week[column_list])

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,...,assists_week27,xPoints week 28,xG_week28,goals_week28,xGA_week28,cleansheet_week28,xA_week28,assists_week28,xPoints week 29,xG_week29,goals_week29,xGA_week29,cleansheet_week29,xA_week29,assists_week29,xPoints week 30,xG_week30,goals_week30,xGA_week30,cleansheet_week30,xA_week30,assists_week30,xPoints week 31,xG_week31,goals_week31,xGA_week31,cleansheet_week31,xA_week31,assists_week31,xPoints week 32,xG_week32,goals_week32,xGA_week32,cleansheet_week32,xA_week32,assists_week32,xPoints week 33,xG_week33,goals_week33,xGA_week33,cleansheet_week33,xA_week33,assists_week33,xPoints week 34,xG_week34,goals_week34,xGA_week34,cleansheet_week34,xA_week34,assists_week34
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0,37605,NaN,,0,0,-3,3,0.0,703,288,NaN,,0,3,0.0,0.0,0,Mesut,0.0,0,0,0.0,703,288,False,0.0,703,288,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,67,0,0,NaN,0,,37605.jpg,0.0,0,0,Özil,0.5,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0.0,0.0,0,39476,NaN,,0,0,-2,2,0.0,664,239,NaN,,0,2,0.0,0.0,0,Sokratis,0.0,0,0,0.0,675,238,False,0.0,672,238,0,Left the club by mutual consent on 20/1,2020-10-21T10:30:18.546407Z,48,0,0,NaN,0,,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,230,75.0,75.0,3,41270,NaN,,0,0,-1,1,46.7,341,115,4.0,,0,2,0.0,1.0,0,David,0.2,20,1,43.9,298,100,False,250.6,253,97,1396,Hamstring injury - 75% chance of playing,2021-05-02T16:00:26.134258Z,54,0,0,NaN,0,,41270.jpg,2.0,1,0,Luiz Moreira Marinho,0.8,False,...,0.0,3.986341,0.0,0.0,0.7,0.0,0.0,0.0,1.401035,0.0,0.0,2.3,0.0,0.0,0.0,NaN,NaN,NaN,2.2,0.0,NaN,NaN,NaN,NaN,NaN,0.4,0.0,NaN,NaN,NaN,NaN,NaN,0.9,0.0,NaN,NaN,NaN,NaN,NaN,0.8,0.0,NaN,NaN,1.6000,0.1,0.0,0.3,0.0,0.0,0.0
4,3,11,350,100.0,100.0,9,54694,NaN,,0,0,-7,7,323.0,111,75,NaN,,3,3,0.0,8.2,6,Pierre-Emerick,3.6,25,10,159.5,50,30,False,489.8,110,40,2162,,2021-04-12T08:30:26.681398Z,113,1,0,1.0,0,,54694.jpg,4.7,0,0,Aubameyang,6.8,False,...,0.0,NaN,NaN,NaN,0.7,0.0,NaN,NaN,2.100259,0.0,0.0,2.3,0.0,0.0,0.0,3.010803,0.0,0.0,2.2,0.0,0.3,0.0,NaN,NaN,NaN,0.4,0.0,NaN,NaN,NaN,NaN,NaN,0.9,0.0,NaN,NaN,NaN,NaN,NaN,0.8,0.0,NaN,NaN,10.5184,1.2,1.0,0.3,1.0,0.0,1.0
5,1,3,125,100.0,100.0,2,58822,NaN,,0,0,-4,4,114.8,238,60,5.0,,0,2,0.0,1.0,0,Cédric,0.0,11,0,29.2,345,123,False,110.8,355,135,744,,2020-09-23T09:00:14.881983Z,46,0,0,NaN,0,,58822.jpg,2.8,0,0,Soares,0.3,False,...,NaN,3.986341,0.0,0.0,0.7,0.0,0.0,0.0,NaN,NaN,NaN,2.3,0.0,NaN,NaN,0.000000,0.0,0.0,2.2,0.0,0.0,0.0,NaN,NaN,NaN,0.4,0.0,NaN,NaN,NaN,NaN,NaN,0.9,0.0,NaN,NaN,NaN,NaN,NaN,0.8,0.0,NaN,NaN,NaN,NaN,NaN,0.3,0.0,NaN,NaN


In [19]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
272,De Bruyne,24.150943,128,5.3,152.898426,6.330950
388,Kane,32.173913,222,6.9,201.575438,6.265183
302,Fernandes,35.820896,240,6.7,222.061075,6.199205
254,Salah,34.193548,212,6.2,200.146946,5.853354
224,Vardy,31.851852,172,5.4,172.027395,5.400860
390,Son,34.062500,218,6.4,183.191739,5.378106
251,Mané,31.702128,149,4.7,166.924825,5.265414
277,Cancelo,26.000000,130,5.0,135.652341,5.217398
39,Hause,4.000000,28,7.0,20.813539,5.203385


In [20]:
# save data
filepath = '../data/fpl/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [21]:
total_xG = playerStats['xG'].sum()
total_goals = df['goals_scored'].sum()
print('Total goals: ' + str(total_goals))
print('Total xG: ' + str(total_xG))
print('goals per xG: ' + str(total_goals/total_xG))

Total goals: 901
Total xG: 924.5999999999999
goals per xG: 0.974475448842743
